In [1]:
import pandas as pd
import pickle as pkl
from tqdm.auto import tqdm
from datetime import timedelta

import warnings

warnings.simplefilter('ignore')

In [2]:
# get all listing histories
lh = pd.read_csv('data/crmls_closings_lh_20240607.csv')
lh['update_transaction'] = pd.to_datetime(lh['update_transaction'], errors='coerce')
lh.head()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
bk_deeds = pd.concat([pd.read_csv('data/bk_raw_deeds0.csv'), pd.read_csv('data/bk_raw_deeds1.csv'), pd.read_csv('data/bk_raw_deeds2.csv')])
print(len(bk_deeds))
bk_deeds['bk_id'] = bk_deeds['dpid'].astype(str) + bk_deeds['recordingdate'].astype(str)
bk_deeds['recordingdate'] = pd.to_datetime(bk_deeds['recordingdate'], format='%Y%m%d', errors='coerce')
bk_deeds.dropna(subset='recordingdate', inplace=True)
bk_deeds.head()

14021811


,dpid,recordingdate,salesprice,standardizedlanduse,propertyfullstreetaddress,propertycityname,propertyzipcode,propertyhousenumber,propertyunitnumber,bk_id
0,60010012587,2015-11-04,90000.0,2012.0,229 BRUSH ST,OAKLAND,94607.0,229,NaN,6001001258720151104
1,60010012587,2016-06-01,205000.0,2012.0,229 BRUSH ST,OAKLAND,94607.0,229,NaN,6001001258720160601
2,60010012588,2016-06-01,205000.0,2012.0,BRUSH ST,OAKLAND,94607.0,NaN,NaN,6001001258820160601
3,60010012588,2015-11-04,90000.0,2012.0,BRUSH ST,OAKLAND,94607.0,NaN,NaN,6001001258820151104
4,60010012590,2004-05-25,410000.0,3010.0,311 OAK ST,OAKLAND,94607.0,311,C1,6001001259020040525


In [ ]:
crmls_closings = pd.concat([pd.read_csv('data/crmls_all_closings0.csv'), pd.read_csv('data/crmls_all_closings1.csv'), 
                            pd.read_csv('data/crmls_all_closings2.csv'), pd.read_csv('data/crmls_all_closings3.csv'), 
                            pd.read_csv('data/crmls_all_closings4.csv'), pd.read_csv('data/crmls_all_closings5.csv')])
print(len(crmls_closings))
crmls_closings['sale_date'] = pd.to_datetime(crmls_closings['sale_date'], errors='coerce')
#crmls_closings.dropna(subset='sale_date',inplace=True)
#crmls_closings['sale_date'] = pd.to_datetime(crmls_closings['sale_date'])
crmls_closings['sale_date']

6543940


0         2024-04-11 04:00:00
1         2023-10-16 04:00:00
2         2023-08-15 04:00:00
3         2023-06-05 04:00:00
4         2022-05-19 04:00:00
                  ...        
1543935   1999-04-15 04:00:00
1543936   2010-08-16 04:00:00
1543937   2024-06-18 04:00:00
1543938   2004-11-19 05:00:00
1543939   2024-06-18 04:01:00
Name: sale_date, Length: 6543940, dtype: datetime64[ns]

In [ ]:
#crmls_closings[~pd.isna(crmls_closings['building_id'])]

In [ ]:
#crmls_closings[~crmls_closings['id'].isin(lh['listing_id'])]

In [ ]:
crmls_closings = crmls_closings.merge(lh, how='left', left_on='id', right_on='listing_id')
crmls_closings['address_check'] = crmls_closings['display_address'].combine_first(crmls_closings['address'])
crmls_closings['zip_check'] = crmls_closings['zip'].combine_first(crmls_closings['zip-2'])
crmls_closings['sale_date'] = crmls_closings['update_transaction'].combine_first(crmls_closings['sale_date'])
crmls_closings['sale_price'] = crmls_closings['price'].combine_first(crmls_closings['sale_price'])
crmls_closings = crmls_closings[['id', 'rebny_id', 'building_id', 'sale_date', 'sale_price', 'address_check', 'zip_check']]
crmls_closings.columns = ['id', 'rebny_id', 'building_id', 'listings_sale_date', 'listings_sale_price', 'address', 'zip']

In [ ]:
crmls_closings.head()

,id,rebny_id,building_id,listings_sale_date,listings_sale_price,address,zip
0,3316490,HD22036424:CRM,NaN,2024-04-11 04:00:00,17000.0,1527 Kittyhawk Street,92274
1,3316491,SR21249835:CRM,NaN,2023-10-16 04:00:00,30000.0,0 Longview Rd s/o Old Homestead Rd,93543
2,3316494,HD22057614:CRM,NaN,2023-08-15 04:00:00,48000.0,0 View Crest,92257
3,3316506,OC20002047:CRM,NaN,2023-06-05 04:00:00,357000.0,0 Morningfield Drive,91709
4,3316514,SR21155219:CRM,NaN,2022-05-19 04:00:00,36000.0,10313 Caribou Lane,90077


In [ ]:
def first_round_match(first_match, off_market_sales, day_threshold: tuple, price_threshold: tuple):
    first_match['match_key'] = [str(x).upper() for x in first_match['address']]
    first_match = first_match.merge(off_market_sales, how='inner', left_on='match_key', right_on='propertyfullstreetaddress')
    #print(f"matches before thresholds: {first_match['id'].nunique()}")
    #first_match['listings_sale_price'] = pd.to_numeric(first_match['listings_sale_price'])
    #first_match['salesprice'] = pd.to_numeric(first_match['salesprice'])
    first_match = first_match.dropna(subset=['listings_sale_date', 'recordingdate', 'salesprice', 'listings_sale_price'])
    if price_threshold[1]:
        if day_threshold[1]:
            first_match = first_match[(abs(first_match['listings_sale_date'] - first_match['recordingdate']) < day_threshold[1])
                              & (abs(first_match['listings_sale_date'] - first_match['recordingdate']) >= day_threshold[0])
                              & (abs((first_match['listings_sale_price'] - first_match['salesprice'])/first_match['listings_sale_price']) < price_threshold[1])
                              & (abs((first_match['listings_sale_price'] - first_match['salesprice'])/first_match['listings_sale_price']) >= price_threshold[0])]
        else:
            first_match = first_match[
                              (abs(first_match['listings_sale_date'] - first_match['recordingdate']) >= day_threshold[0])
                              & (abs((first_match['listings_sale_price'] - first_match['salesprice'])/first_match['listings_sale_price']) < price_threshold[1])
                              & (abs((first_match['listings_sale_price'] - first_match['salesprice'])/first_match['listings_sale_price']) >= price_threshold[0])]
    else:
        if day_threshold[1]:
            first_match = first_match[(abs(first_match['listings_sale_date'] - first_match['recordingdate']) < day_threshold[1])
                              & (abs(first_match['listings_sale_date'] - first_match['recordingdate']) >= day_threshold[0])
                              & (abs((first_match['listings_sale_price'] - first_match['salesprice'])/first_match['listings_sale_price']) >= price_threshold[0])]
        else:
            first_match = first_match[(abs(first_match['listings_sale_date'] - first_match['recordingdate']) >= day_threshold[0])
                              & (abs((first_match['listings_sale_price'] - first_match['salesprice'])/first_match['listings_sale_price']) >= price_threshold[0])]
    return first_match[['id', 'bk_id']]

In [ ]:
map_dict = {'ALLEE': 'ALY', 'ALLEY': 'ALY', 'ALLY': 'ALY', 'ALY': 'ALY',
             'ANEX': 'ANX', 'ANNEX': 'ANX', 'ANNX': 'ANX', 'ANX': 'ANX',
'ARC': 'ARC', 'ARCADE': 'ARC', 'AV': 'AVE', 'AVE': 'AVE', 'AVEN': 'AVE',
'AVENU': 'AVE', 'AVENUE': 'AVE', 'AVN': 'AVE', 'AVNUE': 'AVE', 'BAYOO': 'BYU',
'BAYOU': 'BYU', 'BCH': 'BCH', 'BEACH': 'BCH', 'BEND': 'BND', 'BND': 'BND',
'BLF': 'BLF', 'BLUF': 'BLF', 'BLUFF': 'BLF', 'BLUFFS': 'BLFS', 'BOT': 'BTM',
'BTM': 'BTM', 'BOTTM': 'BTM', 'BOTTOM': 'BTM', 'BLVD': 'BLVD', 'BOUL': 'BLVD',
'BOULEVARD': 'BLVD', 'BOULV': 'BLVD', 'BR': 'BR', 'BRNCH': 'BR','BRANCH': 'BR',
'BRDGE': 'BRG', 'BRG': 'BRG', 'BRIDGE': 'BRG', 'BRK': 'BRK', 'BROOK': 'BRK',
'BROOKS': 'BRKS', 'BURG': 'BG', 'BURGS': 'BGS', 'BYP': 'BYP', 'BYPA': 'BYP',
'BYPAS': 'BYP', 'BYPASS': 'BYP', 'BYPS': 'BYP', 'CAMP': 'CP', 'CP': 'CP',
'CMP': 'CP', 'CANYN': 'CYN', 'CANYON': 'CYN', 'CNYN': 'CYN', 'CAPE': 'CPE',
'CPE': 'CPE', 'CAUSEWAY': 'CSWY', 'CAUSWA': 'CSWY', 'CSWY': 'CSWY',
'CEN': 'CTR', 'CENT': 'CTR', 'CENTER': 'CTR', 'CENTR': 'CTR', 'CENTRE': 'CTR',
'CNTER': 'CTR', 'CNTR': 'CTR', 'CTR': 'CTR', 'CENTERS': 'CTRS', 'CIR': 'CIR',
'CIRC': 'CIR', 'CIRCL': 'CIR', 'CIRCLE': 'CIR', 'CRCL': 'CIR', 'CRCLE': 'CIR',
'CIRCLES': 'CIRS', 'CLF': 'CLF', 'CLIFF': 'CLF', 'CLFS': 'CLFS',
'CLIFFS': 'CLFS','CLB': 'CLB', 'CLUB': 'CLB', 'COMMON': 'CMN',
'COMMONS': 'CMNS', 'COR': 'COR','CORNER': 'COR', 'CORNERS': 'CORS',
'CORS': 'CORS', 'COURSE': 'CRSE', 'CRSE': 'CRSE','COURT': 'CT', 'CT': 'CT',
'COURTS': 'CTS', 'CTS': 'CTS', 'COVE': 'CV', 'CV': 'CV', 'COVES': 'CVS',
 'CREEK': 'CRK', 'CRK': 'CRK', 'CRESCENT': 'CRES', 'CRES': 'CRES',
'CRSENT': 'CRES', 'CRSNT': 'CRES', 'CREST': 'CRST', 'CROSSING': 'XING',
'CRSSNG': 'XING', 'XING': 'XING', 'CROSSROAD': 'XRD', 'CROSSROADS': 'XRDS',
'CURVE': 'CURV', 'DALE': 'DL', 'DL': 'DL', 'DAM': 'DM', 'DM': 'DM',
'DIV': 'DV', 'DIVIDE': 'DV', 'DV': 'DV', 'DVD': 'DV', 'DR': 'DR', 'DRIV': 'DR',
'DRIVE': 'DR', 'DRV': 'DR', 'DRIVES': 'DRS', 'EST': 'EST', 'ESTATE': 'EST',
'ESTATES': 'ESTS', 'ESTS': 'ESTS', 'EXP': 'EXPY', 'EXPR': 'EXPY',
'EXPRESS': 'EXPY', 'EXPRESSWAY': 'EXPY', 'EXPW': 'EXPY', 'EXPY': 'EXPY',
'EXT': 'EXT', 'EXTENSION': 'EXT', 'EXTN': 'EXT', 'EXTNSN': 'EXT',
'EXTS': 'EXTS', 'FALL': 'FALL', 'FALLS': 'FLS', 'FLS': 'FLS', 'FERRY': 'FRY',
'FRRY': 'FRY', 'FRY': 'FRY', 'FIELD': 'FLD', 'FLD': 'FLD', 'FIELDS': 'FLDS',
'FLDS': 'FLDS', 'FLAT': 'FLT', 'FLT': 'FLT', 'FLATS': 'FLTS', 'FLTS': 'FLTS',
'FORD': 'FRD', 'FRD': 'FRD', 'FORDS': 'FRDS', 'FOREST': 'FRST',
'FORESTS': 'FRST', 'FRST': 'FRST', 'FORG': 'FRG', 'FORGE': 'FRG',
'FRG': 'FRG', 'FORGES': 'FRGS', 'FORK': 'FRK', 'FRK': 'FRK', 'FORKS': 'FRKS',
'FRKS': 'FRKS', 'FORT': 'FT', 'FRT': 'FT', 'FT': 'FT', 'FREEWAY': 'FWY',
'FREEWY': 'FWY', 'FRWAY': 'FWY', 'FRWY': 'FWY', 'FWY': 'FWY', 'GARDEN': 'GDN',
'GARDN': 'GDN', 'GRDEN': 'GDN', 'GRDN': 'GDN', 'GARDENS': 'GDNS',
'GDNS': 'GDNS', 'GRDNS': 'GDNS', 'GATEWAY': 'GTWY', 'GATEWY': 'GTWY',
'GATWAY': 'GTWY', 'GTWAY': 'GTWY', 'GTWY': 'GTWY', 'GLEN': 'GLN',
'GLN': 'GLN', 'GLENS': 'GLNS', 'GREEN': 'GRN', 'GRN': 'GRN',
'GREENS': 'GRNS', 'GROV': 'GRV', 'GROVE': 'GRV', 'GRV': 'GRV',
'GROVES': 'GRVS', 'HARB': 'HBR', 'HARBOR': 'HBR', 'HARBR': 'HBR', 'HBR': 'HBR',
'HRBOR': 'HBR', 'HARBORS': 'HBRS', 'HAVEN': 'HVN', 'HVN': 'HVN',
'HT': 'HTS', 'HTS': 'HTS', 'HIGHWAY': 'HWY', 'HIGHWY': 'HWY', 'HIWAY': 'HWY',
'HIWY': 'HWY', 'HWAY': 'HWY', 'HWY': 'HWY', 'HILL': 'HL', 'HL': 'HL',
'HILLS': 'HLS', 'HLS': 'HLS', 'HLLW': 'HOLW', 'HOLLOW': 'HOLW',
'HOLLOWS': 'HOLW', 'HOLW': 'HOLW', 'HOLWS': 'HOLW', 'INLT': 'INLT',
'IS': 'IS', 'ISLAND': 'IS', 'ISLND': 'IS', 'ISLANDS': 'ISS', 'ISLNDS': 'ISS',
'ISS': 'ISS', 'ISLE': 'ISLE', 'ISLES': 'ISLE', 'JCT': 'JCT', 'JCTION': 'JCT',
'JCTN': 'JCT', 'JUNCTION': 'JCT', 'JUNCTN': 'JCT', 'JUNCTON': 'JCT',
'JCTNS': 'JCTS', 'JCTS': 'JCTS', 'JUNCTIONS': 'JCTS', 'KEY': 'KY',
'KY': 'KY', 'KEYS': 'KYS', 'KYS': 'KYS', 'KNL': 'KNL', 'KNOL': 'KNL',
'KNOLL': 'KNL', 'KNLS': 'KNLS', 'KNOLLS': 'KNLS', 'LK': 'LK', 'LAKE': 'LK',
'LKS': 'LKS', 'LAKES': 'LKS', 'LAND': 'LAND', 'LANDING': 'LNDG',
'LNDG': 'LNDG', 'LNDNG': 'LNDG', 'LANE': 'LN', 'LN': 'LN', 'LGT': 'LGT',
'LIGHT': 'LGT', 'LIGHTS': 'LGTS', 'LF': 'LF', 'LOAF': 'LF', 'LCK': 'LCK',
'LOCK': 'LCK', 'LCKS': 'LCKS', 'LOCKS': 'LCKS', 'LDG': 'LDG', 'LDGE': 'LDG',
'LODG': 'LDG', 'LODGE': 'LDG', 'LOOP': 'LOOP', 'LOOPS': 'LOOP', 'MALL': 'MALL',
'MNR': 'MNR', 'MANOR': 'MNR', 'MANORS': 'MNRS', 'MNRS': 'MNRS', 'MEADOW': 'MDW',
'MDW': 'MDWS', 'MDWS': 'MDWS', 'MEADOWS': 'MDWS', 'MEDOWS': 'MDWS',
'MEWS': 'MEWS', 'MILL': 'ML', 'MILLS': 'MLS', 'MISSN': 'MSN', 'MSSN': 'MSN',
'MOTORWAY': 'MTWY', 'MNT': 'MT', 'MT': 'MT', 'MOUNT': 'MT', 'MNTAIN': 'MTN',
'MNTN': 'MTN', 'MOUNTAIN': 'MTN', 'MOUNTIN': 'MTN', 'MTIN': 'MTN',
'MTN': 'MTN', 'MNTNS': 'MTNS', 'MOUNTAINS': 'MTNS', 'NCK': 'NCK',
'NECK': 'NCK', 'ORCH': 'ORCH', 'ORCHARD': 'ORCH', 'ORCHRD': 'ORCH',
'OVAL': 'OVAL', 'OVL': 'OVAL', 'OVERPASS': 'OPAS', 'PARK': 'PARK',
'PRK': 'PARK', 'PARKS': 'PARK', 'PARKWAY': 'PKWY', 'PARKWY': 'PKWY',
'PKWAY': 'PKWY', 'PKWY': 'PKWY', 'PKY': 'PKWY', 'PARKWAYS': 'PKWY',
'PKWYS': 'PKWY', 'PASS': 'PASS', 'PASSAGE': 'PSGE', 'PATH': 'PATH',
'PATHS': 'PATH', 'PIKE': 'PIKE', 'PIKES': 'PIKE', 'PINE': 'PNE',
'PINES': 'PNES', 'PNES': 'PNES', 'PL': 'PL', 'PLAIN': 'PLN', 'PLN': 'PLN',
'PLAINS': 'PLNS', 'PLNS': 'PLNS', 'PLAZA': 'PLZ', 'PLZ': 'PLZ', 'PLZA': 'PLZ',
'POINT': 'PT', 'PT': 'PT', 'POINTS': 'PTS', 'PTS': 'PTS', 'PORT': 'PRT',
'PRT': 'PRT', 'PORTS': 'PRTS', 'PRTS': 'PRTS', 'PR': 'PR', 'PRAIRIE': 'PR',
'PRR': 'PR', 'RAD': 'RADL', 'RADIAL': 'RADL', 'RADIEL': 'RADL',
'RADL': 'RADL', 'RAMP': 'RAMP', 'RANCH': 'RNCH', 'RANCHES': 'RNCH',
'RNCH': 'RNCH', 'RNCHS': 'RNCH', 'RAPID': 'RPD', 'RPD': 'RPD',
'RAPIDS': 'RPDS', 'RPDS': 'RPDS', 'REST': 'RST', 'RST': 'RST', 'RDG': 'RDG',
'RDGE': 'RDG', 'RIDGE': 'RDG', 'RDGS': 'RDGS', 'RIDGES': 'RDGS', 'RIV': 'RIV',
'RIVER': 'RIV', 'RVR': 'RIV', 'RIVR': 'RIV', 'RD': 'RD', 'ROAD': 'RD',
'ROADS': 'RDS', 'RDS': 'RDS', 'ROUTE': 'RTE', 'ROW': 'ROW', 'RUE': 'RUE',
'RUN': 'RUN', 'SHL': 'SHL', 'SHOAL': 'SHL', 'SHLS': 'SHLS', 'SHOALS': 'SHLS',
'SHOAR': 'SHR', 'SHORE': 'SHR', 'SHR': 'SHR', 'SHOARS': 'SHRS',
'SHORES': 'SHRS', 'SHRS': 'SHRS', 'SKYWAY': 'SKWY', 'SPG': 'SPG',
'SPNG': 'SPG', 'SPRING': 'SPG', 'SPRNG': 'SPG', 'SPGS': 'SPGS',
'SPNGS': 'SPGS', 'SPRINGS': 'SPGS', 'SPRNGS': 'SPGS', 'SPUR': 'SPUR',
'SPURS': 'SPUR', 'SQ': 'SQ', 'SQR': 'SQ', 'SQRE': 'SQ', 'SQU': 'SQ',
'SQUARE': 'SQ', 'SQRS': 'SQS', 'SQUARES': 'SQS', 'STA': 'STA',
'STATION': 'STA', 'STATN': 'STA', 'STN': 'STA', 'STRA': 'STRA',
'STRAV': 'STRA', 'STRAVEN': 'STRA', 'STRAVENUE': 'STRA', 'STRAVN': 'STRA',
'STRVN': 'STRA', 'STRVNUE': 'STRA', 'STREAM': 'STRM', 'STREME': 'STRM',
'STRM': 'STRM', 'STREET': 'ST', 'STRT': 'ST', 'ST': 'ST', 'STR': 'ST',
'STREETS': 'STS', 'SMT': 'SMT', 'SUMIT': 'SMT', 'SUMITT': 'SMT',
'SUMMIT': 'SMT', 'TER': 'TER', 'TERR': 'TER', 'TERRACE': 'TER',
'THROUGHWAY': 'TRWY', 'TRACE': 'TRCE', 'TRACES': 'TRCE', 'TRCE': 'TRCE',
'TRACK': 'TRAK', 'TRACKS': 'TRAK', 'TRAK': 'TRAK', 'TRK': 'TRAK',
'TRKS': 'TRAK', 'TRAFFICWAY': 'TRFY', 'TRAIL': 'TRL', 'TRAILS': 'TRL',
'TRL': 'TRL', 'TRLS': 'TRL', 'TRAILER': 'TRLR', 'TRLR': 'TRLR',
'TRLRS': 'TRLR', 'TUNEL': 'TUNL', 'TUNL': 'TUNL', 'TUNLS': 'TUNL',
'TUNNEL': 'TUNL', 'TUNNELS': 'TUNL', 'TUNNL': 'TUNL', 'TRNPK': 'TPKE',
'TURNPIKE': 'TPKE', 'TURNPK': 'TPKE', 'UNDERPASS': 'UPAS', 'UN': 'UN',
'UNION': 'UN', 'UNIONS': 'UNS', 'VALLEY': 'VLY', 'VALLY': 'VLY',
'VLLY': 'VLY', 'VLY': 'VLY', 'VALLEYS': 'VLYS', 'VLYS': 'VLYS',
'VDCT': 'VIA', 'VIA': 'VIA', 'VIADCT': 'VIA', 'VIADUCT': 'VIA',
'VIEW': 'VW', 'VW': 'VW', 'VIEWS': 'VWS', 'VWS': 'VWS', 'VILL': 'VLG',
'VILLAG': 'VLG', 'VILLAGE': 'VLG', 'VILLG': 'VLG', 'VILLIAGE': 'VLG',
'VLG': 'VLG', 'VILLAGES': 'VLGS', 'VLGS': 'VLGS', 'VILLE': 'VL',
'VL': 'VL', 'VIS': 'VIS', 'VIST': 'VIS', 'VISTA': 'VIS', 'VST': 'VIS',
'VSTA': 'VIS', 'WALK': 'WALK', 'WALKS': 'WALK', 'WALL': 'WALL', 'WY': 'WAY',
'WAY': 'WAY', 'WAYS': 'WAYS', 'WELL': 'WL', 'WELLS': 'WLS', 'WLS': 'WLS',
'PLACE': 'PL', 'NORTH': 'N', 'NORTHEAST': 'NE', 'NORTHWEST': 'NW',
'SOUTH': 'S', 'SOUTHEAST': 'SE', 'SOUTHWEST': 'SW', 'EAST': 'E', 'WEST': 'W'}

def standardize_address(address: str) -> str:
    address = address.upper()
    address = address.replace('(', '')
    address = address.replace(')', '')
    address = address.replace('#', '')
    address = address.replace('UNIT', '')
    address = address.replace('STE', '')
    address = address.replace('  ', ' ')
    address_split = address.split()
    for i in range(len(address_split)):
        if address_split[i] in map_dict.keys():
            address_split[i] = map_dict[address_split[i]]
    #for index, row in suffixes.iterrows():
     #   if ' ' + row['Alias'] in address:
      #      address = address.replace(row['Alias'], row['Abbreviation'])
    return ' '.join(address_split)

def second_round_match(second_match, off_market_sales, day_threshold, price_threshold):
    #print('starting second match')
    #suffixes = pd.read_csv(f'{base_fp}/usps street suffixes.csv')
    #suffixes.columns = ['Street Suffix', 'Alias', 'Abbreviation']
    #suffixes.fillna(method='ffill', inplace=True)
    #suffixes = suffixes[['Alias', 'Abbreviation']]
    second_match['normalized_address'] = [standardize_address(str(x)) for x in second_match['address']]
    #second_match['normalized_address'] = second_match['address'].apply(standardize_address(x, suffixes))
    off_market_sales['normalized_address'] = [standardize_address(str(x)) for x in off_market_sales['propertyfullstreetaddress']]
    #print('second match done cleaning')
    second_match = second_match.merge(off_market_sales, how='inner', on='normalized_address')
    #print(f"matches before thresholds: {second_match['id'].nunique()}")
    if price_threshold[1]:
        if day_threshold[1]:
            second_match = second_match[(abs((second_match['listings_sale_price'] - second_match['salesprice'])/second_match['listings_sale_price']) < price_threshold[1])
                                & (abs((second_match['listings_sale_price'] - second_match['salesprice'])/second_match['listings_sale_price']) >= price_threshold[0])
                                & (abs(second_match['listings_sale_date'] - second_match['recordingdate']) < day_threshold[1])
                                & (abs(second_match['listings_sale_date'] - second_match['recordingdate']) >= day_threshold[0])]
        else:
            second_match = second_match[(abs((second_match['listings_sale_price'] - second_match['salesprice'])/second_match['listings_sale_price']) < price_threshold[1])
                                & (abs((second_match['listings_sale_price'] - second_match['salesprice'])/second_match['listings_sale_price']) >= price_threshold[0])
                                & (abs(second_match['listings_sale_date'] - second_match['recordingdate']) >= day_threshold[0])]
    else:
        if day_threshold[1]:
            second_match = second_match[
                                (abs((second_match['listings_sale_price'] - second_match['salesprice'])/second_match['listings_sale_price']) >= price_threshold[0])
                                & (abs(second_match['listings_sale_date'] - second_match['recordingdate']) < day_threshold[1])
                                & (abs(second_match['listings_sale_date'] - second_match['recordingdate']) >= day_threshold[0])]
        else:
            second_match = second_match[
                                (abs((second_match['listings_sale_price'] - second_match['salesprice'])/second_match['listings_sale_price']) >= price_threshold[0])
                                & (abs(second_match['listings_sale_date'] - second_match['recordingdate']) >= day_threshold[0])]
    
    return second_match[['id', 'bk_id']]

In [ ]:
def fuzzy_match(lb_address: str, lb_street_name: str, pw_address: str, listing_id, lb_price, pw_price, lb_date, pw_date, day_threshold, price_threshold): #, lb_zip, pw_zip):
    l = lb_address.strip().split()
    p = pw_address.strip().split()
    m_count = 0

    if not ((l[0] == p[0]) and (str(lb_street_name) in pw_address)):
        return None
    
    if len(l) >= len(p):
        len_address = len(p)
        for i in l:
            if i in p: m_count += 1
    else:
        len_address = len(l)
        for i in p:
            if i in l: m_count += 1
    try:
        if price_threshold[1]:
            if day_threshold[1]:
                if (m_count / len_address >= 0.66) \
                    and (abs((lb_price - pw_price) / pw_price) < price_threshold[1])\
                        and (abs((lb_price - pw_price) / pw_price) >= price_threshold[0])\
                            and (abs(pw_date - lb_date) < day_threshold[1])\
                                and (abs(pw_date - lb_date) >= day_threshold[0]):
                    return listing_id
                else:
                    return None
            else:
                if (m_count / len_address >= 0.66) \
                    and (abs((lb_price - pw_price) / pw_price) < price_threshold[1])\
                        and (abs((lb_price - pw_price) / pw_price) >= price_threshold[0])\
                                and (abs(pw_date - lb_date) >= day_threshold[0]):
                    return listing_id
                else:
                    return None

        else:
            if day_threshold[1]:
                if (m_count / len_address >= 0.66) \
                        and (abs((lb_price - pw_price) / pw_price) >= price_threshold[0])\
                            and (abs(pw_date - lb_date) < day_threshold[1])\
                                and (abs(pw_date - lb_date) >= day_threshold[0]):
                    return listing_id
                else:
                    return None
            else:
                if (m_count / len_address >= 0.66) \
                        and (abs((lb_price - pw_price) / pw_price) >= price_threshold[0])\
                                and (abs(pw_date - lb_date) >= day_threshold[0]):
                    return listing_id
                else:
                    return None
    except ZeroDivisionError:
        return None

def try_to_int(s):
    try:
        if '-' in str(s):
            return int(s.split('-')[0])
        else:
            return int(s)
    except ValueError:
        return None


def third_round_match(third_match, off_market_sales, day_threshold, price_threshold):
    third_match['check_zip'] = [try_to_int(x) for x in third_match['zip']]
    third_match = third_match.dropna(subset='zip')
    off_market_sales['streetname'] = [str(x).replace(str(y), '').strip() if y else str(x) for x, y in zip(off_market_sales['propertyfullstreetaddress'], off_market_sales['propertyhousenumber'])]
    third_match['normalized_address'] = [standardize_address(str(x)) for x in third_match['address']]
    off_market_sales['normalized_address'] = [standardize_address(str(x)) for x in off_market_sales['propertyfullstreetaddress']]
    retlist = []
    for lb_address, lb_price, lb_date, lb_street_name, lb_zip in tqdm(zip(off_market_sales['normalized_address'],
                                                          off_market_sales['salesprice'],
                                                          off_market_sales['recordingdate'], 
                                                          off_market_sales['streetname'], 
                                                          off_market_sales['propertyzipcode']), 
                                                          total=len(off_market_sales), leave=True):
    #for lb_address, lb_price, lb_date, lb_zip in zip(off_market_sales['normalized_address'], off_market_sales['VAL_TRANSFER'], off_market_sales['DATE_TRANSFER'], off_market_sales['SITE_ZIP']):
        #if len(retlist)%10000 == 0:
         #   #print(len(retlist), len(off_market_sales))
        check = third_match[third_match['check_zip'] == lb_zip]
        found = False
        for pw_address, pw_price, pw_date, l_id in zip(check['normalized_address'], check['listings_sale_price'], check['listings_sale_date'], check['id']):
            result = fuzzy_match(lb_address, lb_street_name, pw_address, l_id, lb_price, pw_price, lb_date, pw_date, day_threshold, price_threshold)
            if result:
                found = True
                retlist.append(result)
                break
        if not found:
            retlist.append(None)

    off_market_sales['matched_listing_id'] = retlist
    off_market_sales[~pd.isna(off_market_sales['matched_listing_id'])].merge(third_match, how='inner', left_on='matched_listing_id', right_on='id').to_csv(f'data/fuzzy_matches.csv')
    return off_market_sales[~pd.isna(off_market_sales['matched_listing_id'])][['matched_listing_id', 'bk_id']]

In [ ]:
def threshold_match(pw_closings: pd.DataFrame, lb_deeds: pd.DataFrame,
                     day_threshold: tuple, price_threshold: tuple) -> pd.DataFrame:
    
    matched_pw_ids = []
    matched_lb_ids = []
    pw_closings['check_zip'] = [try_to_int(x) for x in pw_closings['zip']]
    for pw_zip, pw_price, pw_date, pw_id in tqdm(zip(pw_closings['check_zip'], pw_closings['listings_sale_price'], pw_closings['listings_sale_date'], pw_closings['id']), total=len(pw_closings)):
        check = lb_deeds[lb_deeds['propertyzipcode'] == pw_zip]
        found_lb_id = []
        try:
            for lb_price, lb_date, lb_id in zip(check['salesprice'], check['recordingdate'], check['bk_id']):
                    if ((price_threshold[0] <= (abs(pw_price - lb_price)/pw_price) < price_threshold[1]) 
                        and (day_threshold[0] <= abs(pw_date - lb_date) < day_threshold[1])):

                        found_lb_id.append(lb_id)
        
        except (ZeroDivisionError, TypeError):
            continue

        if len(found_lb_id) >= 0:
            matched_pw_ids += [pw_id] * len(found_lb_id)
            matched_lb_ids += found_lb_id
    
    return pd.DataFrame({'id': matched_pw_ids, 'bk_id': matched_lb_ids})

In [ ]:
day_thresh = [timedelta(days=0), timedelta(days=14), timedelta(days=30), timedelta(days=90), timedelta(days=180), timedelta(days=365), timedelta(days=520), None]
price_thresh = [0, 0.1, 0.3, 0.6, 1, None]

In [ ]:
try:
    with open('data/matches_buckets.pkl', 'rb') as f:
        retdict = pkl.load(f)
except:
    retdict = {}
    
with open('data/unmatched_closings.pkl', 'rb') as f:
      crmls_closings = pkl.load(f)
with open('data/unmatched_deeds.pkl', 'rb') as f:
      bk_deeds = pkl.load(f)

In [ ]:
try:
    with open('data/matches_buckets.pkl', 'rb') as f:
        retdict = pkl.load(f)
except:
    retdict = {}
    
with open('data/unmatched_closings.pkl', 'rb') as f:
      crmls_closings = pkl.load(f)
with open('data/unmatched_deeds.pkl', 'rb') as f:
      bk_deeds = pkl.load(f)

In [ ]:
for i in tqdm(range(len(day_thresh)-1)):
    for j in tqdm(range(len(price_thresh)-1),leave=True):
        day_range = (day_thresh[i], day_thresh[i+1])
        price_range = (price_thresh[j], price_thresh[j+1])
        try:
            if type(retdict[(day_range, price_range)]) == pd.DataFrame:
                print((day_range, price_range), retdict[(day_range, price_range)]['id'].nunique())
                pass
        except:
            first_match = first_round_match(crmls_closings, bk_deeds, day_range, price_range)
            first_match['match_type'] = ['1_first']*len(first_match)
            second_match = second_round_match(crmls_closings[~crmls_closings['id'].isin(first_match['id'])],
                                              bk_deeds[~bk_deeds.bk_id.isin(first_match['bk_id'])], 
                                              day_range, 
                                              price_range)
            second_match['match_type'] = ['2_second']*len(second_match)
            matched = pd.concat([first_match, second_match])
            third_match = third_round_match(crmls_closings[~crmls_closings['id'].isin(matched['id'])], 
                                            bk_deeds[~bk_deeds.bk_id.isin(matched['bk_id'])], 
                                            day_range, 
                                            price_range)
            third_match['match_type'] = ['3_third']*len(third_match)
            third_match.columns = ['id', 'bk_id', 'match_type']
            matched = pd.concat([first_match, second_match, third_match])
            retdict[(day_range, price_range)] = matched
            with open('data/matches_buckets.pkl', 'wb') as f:
                pkl.dump(retdict, f)
            crmls_closings = crmls_closings[~crmls_closings['id'].isin(matched['id'])]
            bk_deeds = bk_deeds[~bk_deeds.bk_id.isin(matched['bk_id'])]
            with open('data/unmatched_closings.pkl', 'wb') as f:
                pkl.dump(crmls_closings, f)
            with open('data/unmatched_deeds.pkl', 'wb') as f:
                pkl.dump(bk_deeds, f)
            print(day_range, price_range, first_match['id'].nunique() + second_match['id'].nunique() + third_match['id'].nunique())

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

((datetime.timedelta(0), datetime.timedelta(days=14)), (0, 0.1)) 5027211
((datetime.timedelta(0), datetime.timedelta(days=14)), (0.1, 0.3)) 21861
((datetime.timedelta(0), datetime.timedelta(days=14)), (0.3, 0.6)) 13729
((datetime.timedelta(0), datetime.timedelta(days=14)), (0.6, 1)) 8373
((datetime.timedelta(0), datetime.timedelta(days=14)), (1, None)) 9676


  0%|          | 0/5 [00:00<?, ?it/s]

((datetime.timedelta(days=14), datetime.timedelta(days=30)), (0, 0.1)) 101231
((datetime.timedelta(days=14), datetime.timedelta(days=30)), (0.1, 0.3)) 3048
((datetime.timedelta(days=14), datetime.timedelta(days=30)), (0.3, 0.6)) 1429
((datetime.timedelta(days=14), datetime.timedelta(days=30)), (0.6, 1)) 969
((datetime.timedelta(days=14), datetime.timedelta(days=30)), (1, None)) 767


...

((datetime.timedelta(days=180), datetime.timedelta(days=365)), (0, 0.1)) 6874
((datetime.timedelta(days=180), datetime.timedelta(days=365)), (0.1, 0.3)) 6614
((datetime.timedelta(days=180), datetime.timedelta(days=365)), (0.3, 0.6)) 4515
((datetime.timedelta(days=180), datetime.timedelta(days=365)), (0.6, 1)) 3226
((datetime.timedelta(days=180), datetime.timedelta(days=365)), (1, None)) 2540


  0%|          | 0/5 [00:00<?, ?it/s]

((datetime.timedelta(days=365), datetime.timedelta(days=520)), (0, 0.1)) 2711
((datetime.timedelta(days=365), datetime.timedelta(days=520)), (0.1, 0.3)) 3949
((datetime.timedelta(days=365), datetime.timedelta(days=520)), (0.3, 0.6)) 2903
((datetime.timedelta(days=365), datetime.timedelta(days=520)), (0.6, 1)) 1984
((datetime.timedelta(days=365), datetime.timedelta(days=520)), (1, None)) 1724


  0%|          | 0/5 [00:00<?, ?it/s]

((datetime.timedelta(days=520), None), (0, 0.1)) 46759
((datetime.timedelta(days=520), None), (0.1, 0.3)) 44141
((datetime.timedelta(days=520), None), (0.3, 0.6)) 56531
((datetime.timedelta(days=520), None), (0.6, 1)) 51376
((datetime.timedelta(days=520), None), (1, None)) 66781


In [ ]:
DAY_THRESH = (timedelta(days=0), timedelta(days=14))
PRICE_THRESH = (0, 0.1)
first_match = first_round_match(crmls_closings, bk_deeds, DAY_THRESH, PRICE_THRESH)
second_match = second_round_match(crmls_closings[~crmls_closings['id'].isin(first_match['id'])], bk_deeds[~bk_deeds.bk_id.isin(first_match['bk_id'])], DAY_THRESH, PRICE_THRESH)
matched = pd.concat([first_match, second_match])
third_match = third_round_match(crmls_closings[~crmls_closings['id'].isin(matched['id'])], bk_deeds[~bk_deeds.bk_id.isin(matched['bk_id'])], DAY_THRESH, PRICE_THRESH)
third_match.columns = ['id', 'bk_id', 'match_type']
print(first_match['id'].nunique() + second_match['id'].nunique() + third_match['id'].nunique())

"DAY_THRESH = (timedelta(days=0), timedelta(days=180))\nPRICE_THRESH = (0, None)\nfirst_match = first_round_match(crmls_closings, bk_deeds, DAY_THRESH, PRICE_THRESH)\nsecond_match = second_round_match(crmls_closings[~crmls_closings['id'].isin(first_match['id'])], bk_deeds[~bk_deeds.bk_id.isin(first_match['bk_id'])], DAY_THRESH, PRICE_THRESH)\nmatched = pd.concat([first_match, second_match])\nthird_match = third_round_match(crmls_closings[~crmls_closings['id'].isin(matched['id'])], bk_deeds[~bk_deeds.bk_id.isin(matched['bk_id'])], DAY_THRESH, PRICE_THRESH)\nthird_match.columns = ['id', 'bk_id', 'match_type']\nprint(first_match['id'].nunique() + second_match['id'].nunique() + third_match['id'].nunique())"

In [ ]:
day_thresh = [timedelta(days=0), timedelta(days=7), timedelta(days=14), timedelta(days=30), timedelta(days=60), timedelta(days=90)]
price_thresh = [0, 0.05, 0.1, 0.15]

try:
    with open('data/threshold_matching.pkl', 'rb') as f:
        retdict1 = pkl.load(f)
except:
    retdict1 = {}

In [ ]:
with open('data/unmatched_closings.pkl', 'rb') as f:
      crmls_closings = pkl.load(f)
with open('data/unmatched_deeds.pkl', 'rb') as f:
      bk_deeds = pkl.load(f)

In [ ]:
matched_ids = []
matched_bks = []
for i in tqdm(range(len(day_thresh)-1)):
    for j in tqdm(range(len(price_thresh)-1),leave=True):
        day_range = (day_thresh[i], day_thresh[i+1])
        price_range = (price_thresh[j], price_thresh[j+1])
        try:
            if retdict1[(day_range, price_range)]:
                pass
        except:
            fourth_match = threshold_match(crmls_closings[~crmls_closings['id'].isin(matched_ids)], bk_deeds[~bk_deeds.bk_id.isin(matched_bks)], day_range, price_range)
            fourth_match['match_type'] = ['4_threshold']*len(fourth_match)
            retdict1[(day_range, price_range)] = fourth_match
            matched_ids += list(fourth_match['id'])
            matched_bks += list(fourth_match['bk_id'])
            with open('data/threshold_matching.pkl', 'wb') as f:
                pkl.dump(retdict1, f)
            print(day_range, price_range, fourth_match['id'].nunique())

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/932771 [00:00<?, ?it/s]

(datetime.timedelta(0), datetime.timedelta(days=7)) (0, 0.05) 193863


  0%|          | 0/738908 [00:00<?, ?it/s]

...

(datetime.timedelta(days=60), datetime.timedelta(days=90)) (0, 0.05) 7033


  0%|          | 0/595265 [00:00<?, ?it/s]

(datetime.timedelta(days=60), datetime.timedelta(days=90)) (0.05, 0.1) 4360


  0%|          | 0/590905 [00:00<?, ?it/s]

(datetime.timedelta(days=60), datetime.timedelta(days=90)) (0.1, 0.15) 3826


In [ ]:
bk_deeds = pd.concat([pd.read_csv('data/bk_raw_deeds0.csv'), pd.read_csv('data/bk_raw_deeds1.csv'), pd.read_csv('data/bk_raw_deeds2.csv')])
print(len(bk_deeds))
bk_deeds['bk_id'] = bk_deeds['dpid'].astype(str) + bk_deeds['recordingdate'].astype(str)
bk_deeds['recordingdate'] = pd.to_datetime(bk_deeds['recordingdate'], format='%Y%m%d', errors='coerce')
bk_deeds.dropna(subset='recordingdate', inplace=True)
bk_deeds.head()

14021811


,dpid,recordingdate,salesprice,standardizedlanduse,propertyfullstreetaddress,propertycityname,propertyzipcode,propertyhousenumber,propertyunitnumber,bk_id
0,60010012587,2015-11-04,90000.0,2012.0,229 BRUSH ST,OAKLAND,94607.0,229,NaN,6001001258720151104
1,60010012587,2016-06-01,205000.0,2012.0,229 BRUSH ST,OAKLAND,94607.0,229,NaN,6001001258720160601
2,60010012588,2016-06-01,205000.0,2012.0,BRUSH ST,OAKLAND,94607.0,NaN,NaN,6001001258820160601
3,60010012588,2015-11-04,90000.0,2012.0,BRUSH ST,OAKLAND,94607.0,NaN,NaN,6001001258820151104
4,60010012590,2004-05-25,410000.0,3010.0,311 OAK ST,OAKLAND,94607.0,311,C1,6001001259020040525


In [ ]:
crmls_closings = pd.concat([pd.read_csv('data/crmls_all_closings0.csv'), pd.read_csv('data/crmls_all_closings1.csv'), 
                            pd.read_csv('data/crmls_all_closings2.csv'), pd.read_csv('data/crmls_all_closings3.csv'), 
                            pd.read_csv('data/crmls_all_closings4.csv'), pd.read_csv('data/crmls_all_closings5.csv')])
print(len(crmls_closings))
crmls_closings['sale_date'] = pd.to_datetime(crmls_closings['sale_date'], errors='coerce')
#crmls_closings.dropna(subset='sale_date',inplace=True)
#crmls_closings['sale_date'] = pd.to_datetime(crmls_closings['sale_date'])
#crmls_closings['sale_date']

6543940


In [ ]:
crmls_closings = crmls_closings.merge(lh, how='left', left_on='id', right_on='listing_id')
crmls_closings['address_check'] = crmls_closings['display_address'].combine_first(crmls_closings['address'])
crmls_closings['zip_check'] = crmls_closings['zip'].combine_first(crmls_closings['zip-2'])
crmls_closings['sale_date'] = crmls_closings['update_transaction'].combine_first(crmls_closings['sale_date'])
crmls_closings['sale_price'] = crmls_closings['price'].combine_first(crmls_closings['sale_price'])
crmls_closings = crmls_closings[['id', 'rebny_id', 'building_id', 'sale_date', 'sale_price', 'address_check', 'zip_check']]
crmls_closings.columns = ['id', 'rebny_id', 'building_id', 'listings_sale_date', 'listings_sale_price', 'address', 'zip']

In [ ]:
output = {}
for k in tqdm(retdict, total=len(retdict.keys())):
    bucket = retdict[k].merge(bk_deeds, on='bk_id', how='inner').merge(crmls_closings, on='id', how='inner')[['id', 'bk_id', 'match_type', 'propertyfullstreetaddress', 'propertyunitnumber', 'propertyzipcode', 'salesprice', 'recordingdate', 'building_id', 'address', 'zip', 'listings_sale_price', 'listings_sale_date']]
    try:
        bucket = bucket.sample(10000, random_state=42)
    except:
        pass
    try:
        try:
            output[f'{k[0][0].days}-{k[0][1].days} days,{k[1][0]*100:.0f}%-{k[1][1]*100:.0f}% price'] = bucket
        except:
            output[f'{k[0][0].days}-{k[0][1].days} days,>{k[1][0]*100:.0f}% price'] = bucket
    except:
        try:
            output[f'>{k[0][0].days} days,{k[1][0]*100:.0f}%-{k[1][1]*100:.0f}% price'] = bucket
        except:
            output[f'>{k[0][0].days} days,>{k[1][0]*100:.0f}% price'] = bucket

output1 = {}
for k in tqdm(retdict1, total=len(retdict1.keys())):
    bucket = retdict1[k].merge(bk_deeds, on='bk_id', how='inner').merge(crmls_closings, on='id', how='inner')[['id', 'bk_id', 'match_type', 'propertyfullstreetaddress', 'propertyunitnumber', 'propertyzipcode', 'salesprice', 'recordingdate', 'building_id', 'address', 'zip', 'listings_sale_price', 'listings_sale_date']]
    try:
        bucket = bucket.sample(10000, random_state=42)
    except:
        pass
    try:
        try:
            output1[f'TM_{k[0][0].days}-{k[0][1].days} days,{k[1][0]*100:.0f}%-{k[1][1]*100:.0f}% price'] = bucket
        except:
            output1[f'TM_{k[0][0].days}-{k[0][1].days} days,>{k[1][0]*100:.0f}% price'] = bucket
    except:
        try:
            output1[f'TM_>{k[0][0].days} days,{k[1][0]*100:.0f}%-{k[1][1]*100:.0f}% price'] = bucket
        except:
            output1[f'TM_>{k[0][0].days} days,>{k[1][0]*100:.0f}% price'] = bucket

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
def report_writer(dfs:dict, report_name:str):
    """Creates an excel report using a dictionary of dfs.

    Args:
        dfs (dict): Data
        report_name (str): Name of file.
    """
    writer = pd.ExcelWriter(report_name, engine='xlsxwriter')
    workbook = writer.book
    format = workbook.add_format()
    format.set_align('center')
    for sheetname, df in dfs.items():
        df.to_excel(writer, sheet_name=sheetname, index=False)
        worksheet = writer.sheets[sheetname]
        for idx, col in enumerate(df):
            series =df[col]
            max_len = max((
                series.astype(str).map(len).max(),
                len(str(series.name))
            ))+1
            worksheet.set_column(idx, idx, max_len, format)

    writer.close()

In [ ]:
len(output.keys())

35

In [ ]:
report_writer(output, 'data/output/matches_by_bucket.xlsx')
report_writer(output1, 'data/output/threshold_matches_by_bucket.xlsx')

In [ ]:
bk_deeds[bk_deeds['bk_id'].isin(retdict[(timedelta(0), timedelta(days=14)), (0, 0.1)]['bk_id'])].to_csv('data/output/deeds_to_hide_0_14.csv', index=False)

In [ ]:
bk_deeds[bk_deeds['bk_id'].isin(retdict[(timedelta(14), timedelta(days=30)), (0, 0.1)]['bk_id'])].to_csv('data/output/deeds_to_hide_14_30.csv', index=False)

In [ ]:
bk_deeds[bk_deeds['bk_id'].isin(retdict[(timedelta(30), timedelta(days=90)), (0, 0.1)]['bk_id'])].to_csv('data/output/deeds_to_hide_30_90.csv', index=False)